In [2]:
import pandas as pd

# Read CSV files into pandas DataFrames
stations_dynamic_df = pd.read_csv("ExportedData/stations_dynamic.csv")
stations_static_df = pd.read_csv("ExportedData/stations_static.csv")
weather_data_df = pd.read_csv("ExportedData/weather_data.csv")

# Display the first few rows of each DataFrame to understand the data structure
print("Stations Dynamic Data:")
print(stations_dynamic_df.head())
print("\nStations Static Data:")
print(stations_static_df.head())
print("\nWeather Data:")
print(weather_data_df.head())

# Perform some simple analysis
# Example: Calculate the average temperature from weather data
average_temperature = weather_data_df['temperature'].mean()
print("\nAverage Temperature:", average_temperature)

# Example: Count the number of bike stations
num_stations = len(stations_static_df)
print("Number of Bike Stations:", num_stations)

# Example: Find the most common main events in weather data
common_main_events = weather_data_df['main_event'].value_counts().head()
print("\nMost Common Main Events:")
print(common_main_events)


Stations Dynamic Data:
   id  number                  name  banking  bonus status  \
0   1      42      SMITHFIELD NORTH        0      0   OPEN   
1   2      30  PARNELL SQUARE NORTH        0      0   OPEN   
2   3      54        CLONMEL STREET        0      0   OPEN   
3   4     108         AVONDALE ROAD        0      0   OPEN   
4   5      20     JAMES STREET EAST        0      0   OPEN   

           last_update           api_update  available_bikes  \
0  2024-02-21 00:59:33  2024-02-21 01:00:25               30   
1  2024-02-21 00:51:51  2024-02-21 01:00:25                0   
2  2024-02-21 00:50:28  2024-02-21 01:00:25                3   
3  2024-02-21 00:51:03  2024-02-21 01:00:25               25   
4  2024-02-21 00:52:14  2024-02-21 01:00:25                0   

   available_bike_stands  bike_stands  
0                      0           30  
1                     20           20  
2                     30           33  
3                     10           35  
4                  